The final version of my decision.

Many thanks to the guys whose public kernels helped create this script - @zxspectrum and @ yag320

In [1]:
import math
import pandas as pd
from itertools import islice
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import logging
from tqdm import tqdm
import category_encoders
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, scale, MinMaxScaler, Normalizer, QuantileTransformer, PowerTransformer, StandardScaler
from scipy.stats import boxcox
import math
from sklearn.preprocessing import KBinsDiscretizer
from catboost import Pool, CatBoostClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
test_path = '../input/test.csv'

df_test = pd.read_csv(test_path)
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in tqdm(range(df_test.shape[1])):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

del df_test

100%|██████████| 200/200 [00:06<00:00, 32.83it/s]


In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

test_real = test.iloc[real_samples_indexes]

features = train.drop(['ID_code', 'target'], axis = 1).columns.tolist()

data = train.append(test_real)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
num_round = 1000000
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))

model = CatBoostClassifier(loss_function="Logloss",
                           eval_metric="AUC",
                           task_type="GPU",
                           learning_rate=0.01,
                           iterations=70000,
                           l2_leaf_reg=50,
                           random_seed=42,
                           od_type="Iter",
                           depth=5,
                           early_stopping_rounds=15000,
                           border_count=64
                          )

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train.target.values)):
    print("Fold {}".format(fold_))
    X_train, y_train = train.iloc[trn_idx][features], train.iloc[trn_idx]['target']
    X_valid, y_valid = train.iloc[val_idx][features], train.iloc[val_idx]['target']
    
    for col in tqdm(features):
        gr = data[col].value_counts()
        gr_bin = data.groupby(col)[col].count()>1
        
        X_train[col + '_un'] = X_train[col].map(gr).astype('category').cat.codes
        X_valid[col + '_un'] = X_valid[col].map(gr).astype('category').cat.codes
        test[col + '_un'] = test[col].map(gr).astype('category').cat.codes
        
        X_train[col + '_un_bin'] = X_train[col].map(gr_bin).astype('category').cat.codes
        X_valid[col + '_un_bin'] = X_valid[col].map(gr_bin).astype('category').cat.codes
        test[col + '_un_bin'] = test[col].map(gr_bin).astype('category').cat.codes
        
        X_train[col + '_raw_mul'] = X_train[col] * X_train[col + '_un_bin']
        X_valid[col + '_raw_mul'] = X_valid[col] * X_valid[col + '_un_bin']
        test[col + '_raw_mul'] = test[col] * test[col + '_un_bin']
        
        X_train[col + '_raw_mul_2'] = X_train[col] * X_train[col + '_un']
        X_valid[col + '_raw_mul_2'] = X_valid[col] * X_valid[col + '_un']
        test[col + '_raw_mul_2'] = test[col] * test[col + '_un']
        
        X_train[col + '_raw_mul_3'] = X_train[col + '_un_bin'] * X_train[col + '_un']
        X_valid[col + '_raw_mul_3'] = X_valid[col + '_un_bin'] * X_valid[col + '_un']
        test[col + '_raw_mul_3'] = test[col + '_un_bin'] * test[col + '_un']


    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    clf = model.fit(_train,
                    eval_set=_valid,
                    use_best_model=True,
                    verbose=5000)
    pred = clf.predict_proba(X_valid)[:,1]
    oof[val_idx] = pred
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    predictions += clf.predict_proba(test.drop('ID_code', axis=1))[:,1] / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(train.target, oof)))

Fold 0


100%|██████████| 200/200 [00:54<00:00,  2.43it/s]


0:	learn: 0.5728201	test: 0.5700304	best: 0.5700304 (0)	total: 25.2ms	remaining: 29m 23s
5000:	learn: 0.9240083	test: 0.9066963	best: 0.9066982 (4999)	total: 1m 19s	remaining: 17m 14s
10000:	learn: 0.9398341	test: 0.9137103	best: 0.9137103 (10000)	total: 2m 41s	remaining: 16m 11s
15000:	learn: 0.9485082	test: 0.9158077	best: 0.9158078 (14989)	total: 4m 2s	remaining: 14m 49s
20000:	learn: 0.9548572	test: 0.9167625	best: 0.9167631 (19996)	total: 5m 20s	remaining: 13m 21s
25000:	learn: 0.9599200	test: 0.9172564	best: 0.9172646 (24922)	total: 6m 41s	remaining: 12m 2s
30000:	learn: 0.9643125	test: 0.9175897	best: 0.9175897 (30000)	total: 8m 1s	remaining: 10m 42s
35000:	learn: 0.9681845	test: 0.9179211	best: 0.9179446 (34534)	total: 9m 21s	remaining: 9m 21s
40000:	learn: 0.9717070	test: 0.9180931	best: 0.9180935 (39994)	total: 10m 41s	remaining: 8m 1s
45000:	learn: 0.9748462	test: 0.9182405	best: 0.9182447 (44974)	total: 12m 1s	remaining: 6m 40s
50000:	learn: 0.9777141	test: 0.9183286	best: 

100%|██████████| 200/200 [00:40<00:00,  2.50it/s]


0:	learn: 0.5807576	test: 0.5756657	best: 0.5756657 (0)	total: 19.5ms	remaining: 22m 47s
5000:	learn: 0.9235764	test: 0.9086463	best: 0.9086494 (4998)	total: 1m 20s	remaining: 17m 22s
10000:	learn: 0.9397370	test: 0.9150954	best: 0.9150954 (10000)	total: 2m 40s	remaining: 16m 5s
15000:	learn: 0.9483536	test: 0.9166453	best: 0.9166482 (14981)	total: 4m 4s	remaining: 14m 56s
20000:	learn: 0.9546004	test: 0.9173993	best: 0.9174038 (19976)	total: 5m 25s	remaining: 13m 33s
25000:	learn: 0.9597131	test: 0.9177096	best: 0.9177149 (24980)	total: 6m 45s	remaining: 12m 9s
30000:	learn: 0.9640281	test: 0.9178697	best: 0.9178697 (30000)	total: 8m 5s	remaining: 10m 47s
35000:	learn: 0.9678140	test: 0.9180399	best: 0.9180449 (34965)	total: 9m 26s	remaining: 9m 26s
40000:	learn: 0.9712778	test: 0.9181550	best: 0.9181579 (39934)	total: 10m 47s	remaining: 8m 5s
45000:	learn: 0.9744183	test: 0.9182366	best: 0.9182383 (44837)	total: 12m 5s	remaining: 6m 43s
50000:	learn: 0.9772736	test: 0.9182865	best: 0

...

0:	learn: 0.6017445	test: 0.5971054	best: 0.5971054 (0)	total: 18.6ms	remaining: 21m 44s
5000:	learn: 0.9224496	test: 0.9124308	best: 0.9124315 (4999)	total: 1m 21s	remaining: 17m 34s
10000:	learn: 0.9384384	test: 0.9197241	best: 0.9197249 (9995)	total: 2m 41s	remaining: 16m 8s
15000:	learn: 0.9471549	test: 0.9216832	best: 0.9216865 (14970)	total: 4m 4s	remaining: 14m 54s
20000:	learn: 0.9534275	test: 0.9227067	best: 0.9227085 (19996)	total: 5m 25s	remaining: 13m 32s
25000:	learn: 0.9585813	test: 0.9232370	best: 0.9232433 (24927)	total: 6m 43s	remaining: 12m 6s
30000:	learn: 0.9630053	test: 0.9235727	best: 0.9235772 (29970)	total: 8m	remaining: 10m 41s
35000:	learn: 0.9669683	test: 0.9237240	best: 0.9237244 (34996)	total: 9m 19s	remaining: 9m 19s
40000:	learn: 0.9705217	test: 0.9238641	best: 0.9238641 (40000)	total: 10m 39s	remaining: 7m 59s
45000:	learn: 0.9737642	test: 0.9239857	best: 0.9239879 (44990)	total: 11m 58s	remaining: 6m 38s
50000:	learn: 0.9767520	test: 0.9240859	best: 0.9

100%|██████████| 200/200 [00:35<00:00,  2.85it/s]


0:	learn: 0.5848160	test: 0.5736620	best: 0.5736620 (0)	total: 18.5ms	remaining: 21m 37s
5000:	learn: 0.9236907	test: 0.9052941	best: 0.9052953 (4999)	total: 1m 20s	remaining: 17m 30s
10000:	learn: 0.9394074	test: 0.9127904	best: 0.9127904 (10000)	total: 2m 39s	remaining: 15m 58s
15000:	learn: 0.9479378	test: 0.9150604	best: 0.9150628 (14961)	total: 3m 58s	remaining: 14m 35s
20000:	learn: 0.9541585	test: 0.9161696	best: 0.9161696 (19999)	total: 5m 16s	remaining: 13m 12s
25000:	learn: 0.9592273	test: 0.9169340	best: 0.9169341 (24999)	total: 6m 34s	remaining: 11m 50s
30000:	learn: 0.9635550	test: 0.9174552	best: 0.9174589 (29937)	total: 7m 52s	remaining: 10m 29s
35000:	learn: 0.9675671	test: 0.9177881	best: 0.9177925 (34980)	total: 9m 11s	remaining: 9m 11s
40000:	learn: 0.9710876	test: 0.9180076	best: 0.9180156 (39812)	total: 10m 30s	remaining: 7m 52s
45000:	learn: 0.9743321	test: 0.9181750	best: 0.9181761 (44993)	total: 11m 48s	remaining: 6m 33s
50000:	learn: 0.9772437	test: 0.9182649	b

100%|██████████| 200/200 [00:34<00:00,  2.87it/s]


0:	learn: 0.5787821	test: 0.5719253	best: 0.5719253 (0)	total: 16.8ms	remaining: 19m 32s
5000:	learn: 0.9243198	test: 0.9042088	best: 0.9042088 (5000)	total: 1m 17s	remaining: 16m 45s
10000:	learn: 0.9402682	test: 0.9122207	best: 0.9122207 (10000)	total: 2m 35s	remaining: 15m 30s
15000:	learn: 0.9489580	test: 0.9144725	best: 0.9144725 (15000)	total: 3m 52s	remaining: 14m 13s
20000:	learn: 0.9552417	test: 0.9155385	best: 0.9155385 (20000)	total: 5m 11s	remaining: 12m 59s
25000:	learn: 0.9602182	test: 0.9161453	best: 0.9161453 (25000)	total: 6m 30s	remaining: 11m 43s
30000:	learn: 0.9645258	test: 0.9165321	best: 0.9165326 (29981)	total: 7m 48s	remaining: 10m 24s
35000:	learn: 0.9684312	test: 0.9168442	best: 0.9168453 (34987)	total: 9m 4s	remaining: 9m 4s
40000:	learn: 0.9719623	test: 0.9170555	best: 0.9170557 (39998)	total: 10m 20s	remaining: 7m 45s
45000:	learn: 0.9751115	test: 0.9171372	best: 0.9171395 (44985)	total: 11m 36s	remaining: 6m 27s
50000:	learn: 0.9779506	test: 0.9172855	bes

In [5]:
sub = pd.DataFrame({"ID_code": test.ID_code.values})
sub["target"] = predictions
sub.to_csv("Range_bins_sub_3.csv", index=False)

In [6]:
sub.head()

,ID_code,target
0,test_0,0.048964
1,test_1,0.179397
2,test_2,0.261497
3,test_3,0.158345
4,test_4,0.053507
